In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import os
import glob
import math
from functools import reduce
import operator as op
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

dataset_folder = os.path.abspath("./individual_npzs/{0}/*.npz")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## Parameters

In [2]:
learning_rate = 0.0001
batch_size = 32
dropout = 0.75
max_pool = 2
strides = 1
input_size = 60000
output_size = 4
epochs = 30
timesteps = 38
seed_num = 72
hidden_layer = 64
l2_beta = 0.01

validation_session = 2
test_session = 1

label_dictionary = {'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3}
onehot_dictionary = {0: 'ang', 1: 'hap', 2:'neu', 3:'sad'}

tf.set_random_seed(seed_num)

## Helper methods

In [3]:
def batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    for batch_step in range(0, steps):
        start = batch_size * batch_step
        end = batch_size * (batch_step + 1)
        yield data[start:end], labels[start:end], batch_step
        
def build_encoded_array(emotion_label):
    initialized_array = [0. for key in label_dictionary]
    initialized_array[label_dictionary[emotion_label]] = 1.
    return initialized_array
        
def onehot_encode(label_minibatch):
    return [build_encoded_array(emotion_label) for emotion_label in label_minibatch]

def compute_class_weights(labels):
    return np.ndarray.tolist(class_weight.compute_class_weight('balanced', np.unique(labels), labels))

def sparse_encode(label_minibatch):
    return [label_dictionary[emotion_label] for emotion_label in label_minibatch]

def onehot_decode(onehot_labels):
    return [onehot_dictionary[label.index(1.0)] for label in onehot_labels]

## Load dataset: 3 sessions for training, 1 for validation, 1 for test

In [4]:
train_dataset = []
validation_dataset = []
test_dataset = []

train_labels = []
validation_labels = []
test_labels = []

all_sessions = {1: [], 2:[], 3:[], 4:[], 5:[]}

session_string = 'session{0}'

for i in range(1, 6):
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        for x in loaded_spec['spectrograms']:
            if i != validation_session and i != test_session:
                train_dataset.append(x) 
            elif i == validation_session:
                validation_dataset.append(x)
            elif i == test_session:
                test_dataset.append(x)
        for x in loaded_spec['labels']:
            all_sessions[i].append(x)
            if i != validation_session and i != test_session:
                train_labels.append(x) 
            elif i == validation_session:
                validation_labels.append(x)
            elif i == test_session:
                test_labels.append(x)
        
train_dataset = np.asarray(train_dataset)
train_labels = np.asarray(train_labels)

validation_dataset = np.asarray(validation_dataset)
validation_labels = np.asarray(validation_labels)

test_dataset = np.asarray(test_dataset)
test_labels = np.asarray(test_labels)

In [5]:
[print("Session #{0}".format(session), np.unique(all_sessions[session], return_counts=True)) for session in all_sessions]

Session #1 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([ 84,  46, 275, 173], dtype=int64))
Session #2 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([ 48,  81, 422, 193], dtype=int64))
Session #3 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([124,  82, 282, 258], dtype=int64))
Session #4 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([140,  60, 248, 175], dtype=int64))
Session #5 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([ 56, 157, 432, 221], dtype=int64))


[None, None, None, None, None]

In [6]:
train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(train_dataset)):
    train_data[data,:,:] = train_dataset[data]
    
validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(validation_dataset)):
    validation_data[data,:,:] = validation_dataset[data]
    
test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(test_dataset)):
    test_data[data,:,:] = test_dataset[data]

In [7]:
#class_weights = compute_class_weights(train_labels)
counts = np.unique(train_labels, return_counts=True)[1]
sum_counts = sum(counts)
class_weights = [count/sum_counts for count in counts]
#class_weights = list(map(lambda x: x if x > 1.0 else 1.0, class_weights))

In [8]:
class_weights

[0.14317673378076062,
 0.1337807606263982,
 0.4304250559284116,
 0.29261744966442954]

In [9]:
validation_data = validation_data.reshape((validation_data.shape[0], input_size))
validation_labels = onehot_encode(validation_labels)

test_data = test_data.reshape((test_data.shape[0], input_size))
test_labels = onehot_encode(test_labels)

train_data = train_data.reshape((train_data.shape[0], input_size))
train_labels = onehot_encode(train_labels)

In [10]:
train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_num)

## Convolutional

In [11]:
def conv2d(to_process, weights, biases, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    relu_out = tf.nn.relu(bias_out)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def nn_pipeline(spectrogram, weights, biases):
    
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])
    
    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)
    
    second_layer_out = conv2d(first_maxpool_out, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)
    
    third_layer_out = conv2d(second_maxpool_out, weights['third_layer_weights'], biases['third_layer_biases'])
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)
    
    reshape_for_fc = tf.reshape(third_maxpool_out, [-1, weights['fully_connected_weights'].get_shape().as_list()[0]])
    fully_connected_out = tf.add(tf.matmul(reshape_for_fc, weights['fully_connected_weights']), biases['fully_connected_biases'])
    fully_connected_activation = tf.nn.relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout)
    
    fully_connected_out_2 = tf.add(tf.matmul(fully_connected_dropout, weights['fully_connected_weights_2']), biases['fully_connected_biases_2'])
    fully_connected_activation_2 = tf.nn.relu(fully_connected_out_2)
    fully_connected_dropout_2 = tf.nn.dropout(fully_connected_activation_2, dropout)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout_2, weights['output']), biases['output'])
    
    return prediction

## Testing

In [12]:
def nn_pipeline_rnn(spectrogram, weights, biases, dropout_num, normalization_switch):
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])

    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)
    first_batch_norm = tf.layers.batch_normalization(first_maxpool_out, training=normalization_switch)

    second_layer_out = conv2d(first_batch_norm, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)
    second_batch_norm = tf.layers.batch_normalization(second_maxpool_out, training=normalization_switch)

    third_layer_out = conv2d(second_batch_norm, weights['third_layer_weights'], biases['third_layer_biases'])
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)
    third_batch_norm = tf.layers.batch_normalization(third_maxpool_out, training=normalization_switch)

    interim_shape = third_batch_norm.get_shape().as_list()
    transposed = tf.transpose(third_maxpool_out, perm=[0, 2, 1, 3])
    reshape_for_rnn = tf.reshape(transposed, [-1, interim_shape[2], interim_shape[1]*interim_shape[3]])
    reshape_for_rnn.set_shape([None, interim_shape[2], interim_shape[1]*interim_shape[3]])

    hidden_list = [hidden_layer, hidden_layer]

    gru_fw_cell = [tf.contrib.rnn.GRUCell(hidden) for hidden in hidden_list]
    gru_bw_cell = [tf.contrib.rnn.GRUCell(hidden) for hidden in hidden_list]

    gru_output, _, _, = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    interim_shape_gru = tf.shape(gru_output)
    gru_flatten = tf.reshape(gru_output, [-1, interim_shape_gru[1]*interim_shape_gru[2]])
    
    fully_connected_out = tf.add(tf.matmul(gru_flatten, weights['gru_weights']), biases['gru_biases'])
    fully_connected_activation = tf.nn.relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout_num)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout, weights['output']), biases['output'])
        
    return prediction

In [13]:
weights = {
    'first_layer_weights': tf.Variable(tf.random_normal([10, 15, 1, 16])),
    'second_layer_weights': tf.Variable(tf.random_normal([8, 10, 16, 24])),
    'third_layer_weights': tf.Variable(tf.random_normal([5, 8, 24, 32])),
    'fully_connected_weights': tf.Variable(tf.random_normal([25*38*32, 2048])),
    'fully_connected_weights_2': tf.Variable(tf.random_normal([2048, 2048])),
    'output': tf.Variable(tf.random_normal([2048, output_size]))
}

biases = {
    'first_layer_biases': tf.Variable(tf.random_normal([16])),
    'second_layer_biases': tf.Variable(tf.random_normal([24])),
    'third_layer_biases': tf.Variable(tf.random_normal([32])),
    'fully_connected_biases': tf.Variable(tf.random_normal([2048])),
    'fully_connected_biases_2': tf.Variable(tf.random_normal([2048])),
    'output': tf.Variable(tf.random_normal([output_size]))
}

In [14]:
weights_rnn = {
    'first_layer_weights': tf.Variable(tf.truncated_normal([10, 15, 1, 16], seed=seed_num)),
    'second_layer_weights': tf.Variable(tf.truncated_normal([8, 10, 16, 24], seed=seed_num)),
    'third_layer_weights': tf.Variable(tf.truncated_normal([5, 8, 24, 32], seed=seed_num)),
    'gru_weights': tf.Variable(tf.truncated_normal([2*hidden_layer*timesteps, hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([hidden_layer, output_size], seed=seed_num))
}

biases_rnn = {
    'first_layer_biases': tf.Variable(tf.truncated_normal([16], seed=seed_num)),
    'second_layer_biases': tf.Variable(tf.truncated_normal([24], seed=seed_num)),
    'third_layer_biases': tf.Variable(tf.truncated_normal([32], seed=seed_num)),
    'gru_biases': tf.Variable(tf.truncated_normal([hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([output_size], seed=seed_num))
}

In [15]:
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

keep_prob = tf.placeholder(tf.float32)
normalization_switch = tf.placeholder(tf.bool)

logits = nn_pipeline_rnn(X, weights_rnn, biases_rnn, keep_prob, normalization_switch)
prediction = tf.nn.softmax(logits)

weighted_logits = tf.multiply(class_weights, logits)
#weight_regularizer = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * 0.01

loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=weighted_logits, labels=Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_trigger = optimizer.minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [16]:
with tf.device('/GPU:0'):
    sess = tf.Session()
    sess.run(init)

    print('Session initialized.')

    for epoch_step in range(1, epochs+1):
        batch_gen = batch_generator(train_data, train_labels)
        for data_minibatch, label_minibatch, current_index in batch_gen:
            sess.run(train_trigger, feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: dropout, normalization_switch: True})
            #train_loss, train_acc = sess.run([loss_function, accuracy], feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: 1.0, normalization_switch: False})
            #print("Training accuracy of batch #{0}".format(current_index) + ": Loss={:.4f}".format(train_loss) +", Accuracy={:.4f}.".format(train_acc))
        loss, acc = sess.run([loss_function, accuracy], feed_dict={X: validation_data, Y: validation_labels, keep_prob: 1.0, normalization_switch: False})
        print("Validation after epoch #" + str(epoch_step) + ", Validation Loss= "+ "{:.4f}".format(loss) + ", Validation Accuracy= " + "{:.3f}".format(acc))
        if (epoch_step % 5 == 0):
            preds = sess.run(tf.argmax(prediction, 1), feed_dict={X:validation_data, Y:validation_labels, keep_prob:1.0, normalization_switch: False})
            print(preds)
        
    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data,
                                      Y: test_labels,
                                      keep_prob: 1.0, normalization_switch: False}))

Session initialized.
Validation after epoch #1, Validation Loss= 9.9872, Validation Accuracy= 0.442
Validation after epoch #2, Validation Loss= 3.9379, Validation Accuracy= 0.335
Validation after epoch #3, Validation Loss= 2.5752, Validation Accuracy= 0.309
Validation after epoch #4, Validation Loss= 2.0761, Validation Accuracy= 0.304
Validation after epoch #5, Validation Loss= 1.8333, Validation Accuracy= 0.296
[1 1 1 0 0 2 0 3 0 2 0 1 0 3 3 1 1 0 2 0 3 2 2 2 0 2 2 0 3 0 3 0 0 3 3 0 3
 2 0 1 1 0 0 0 2 1 3 2 3 1 1 2 3 3 3 3 3 0 1 1 0 2 0 2 0 0 2 0 1 0 2 3 0 0
 1 2 3 0 3 1 0 0 0 1 1 2 1 3 1 1 0 0 2 3 3 1 0 2 0 2 2 1 0 2 1 0 0 2 2 1 3
 2 0 3 0 2 1 1 3 1 3 0 3 2 1 3 0 3 2 1 2 2 0 2 3 1 1 0 2 2 0 0 1 0 0 0 1 0
 3 0 0 0 0 2 1 2 0 0 0 2 2 0 2 2 2 2 0 0 3 1 3 1 3 2 0 2 1 3 0 2 3 3 1 0 0
 2 1 0 0 0 0 3 0 0 1 0 1 2 3 2 3 0 0 0 0 3 1 2 2 3 2 2 1 1 0 1 1 2 3 0 2 0
 2 3 2 2 0 2 1 3 3 0 1 1 3 0 1 0 3 3 1 2 1 2 2 1 3 1 3 0 3 2 1 0 3 3 2 3 2
 1 2 2 2 1 3 1 3 1 2 2 0 1 3 3 2 1 0 3 0 2 3 0 0 0 1 3 2 3 

Validation after epoch #26, Validation Loss= 1.6109, Validation Accuracy= 0.352
Validation after epoch #27, Validation Loss= 1.6192, Validation Accuracy= 0.375
Validation after epoch #28, Validation Loss= 1.6145, Validation Accuracy= 0.362
Validation after epoch #29, Validation Loss= 1.6250, Validation Accuracy= 0.367
Validation after epoch #30, Validation Loss= 1.6228, Validation Accuracy= 0.359
[1 1 1 2 0 2 0 3 0 2 0 2 0 3 3 1 2 2 0 2 3 2 2 2 0 2 2 0 0 2 3 1 3 3 3 3 3
 2 3 3 1 2 0 0 2 3 2 2 3 2 3 2 3 3 0 3 3 2 3 2 0 0 0 0 0 3 0 0 1 0 2 3 1 0
 2 2 3 0 3 1 0 0 3 0 3 3 1 3 1 1 0 0 2 2 3 1 2 2 0 2 2 3 2 2 1 2 0 2 3 2 3
 2 0 3 0 2 3 3 2 1 3 0 3 2 3 1 2 2 2 1 2 2 2 2 3 1 1 0 1 2 1 0 1 2 0 0 1 3
 3 0 2 2 0 2 1 2 0 3 2 2 2 0 2 2 2 2 0 0 3 1 3 1 3 2 0 3 2 3 0 3 0 3 1 2 0
 3 3 0 0 2 0 2 0 3 1 0 2 2 2 3 3 0 0 0 0 3 1 2 3 3 3 3 1 2 3 1 2 1 2 2 2 3
 3 3 2 2 0 3 1 3 3 0 3 3 3 0 3 0 3 3 1 2 1 2 2 1 3 3 3 2 3 1 1 3 3 3 3 3 3
 3 2 2 2 2 3 3 3 3 3 2 2 3 3 3 3 3 0 3 3 2 3 3 0 0 1 3 2 2 2 2 3 2 3 2 2 2


In [17]:
predictions_test = sess.run(correct_prediction, feed_dict={X:test_data, Y:test_labels, keep_prob:1.0, normalization_switch: False})
predictions_validation = sess.run(correct_prediction, feed_dict={X:validation_data, Y:validation_labels, keep_prob:1.0, normalization_switch: False})

In [18]:
coupled_test = list(zip(predictions_test, onehot_decode(test_labels)))
final_dict_test = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
overall_dict_test = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

for couple in coupled_test:
    if (couple[0]):
        final_dict_test[couple[1]] += 1
    overall_dict_test[couple[1]] += 1
print("Final: ", final_dict_test)
print("Overall: ", overall_dict_test)

Final:  {'ang': 24, 'hap': 5, 'neu': 97, 'sad': 89}
Overall:  {'ang': 84, 'hap': 46, 'neu': 275, 'sad': 173}


In [19]:
coupled_validation = list(zip(predictions_validation, onehot_decode(validation_labels)))
final_dict_validation = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
overall_dict_validation = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

for couple in coupled_validation:
    if (couple[0]):
        final_dict_validation[couple[1]] += 1
    overall_dict_validation[couple[1]] += 1
print("Final: ", final_dict_validation)
print("Overall: ", overall_dict_validation)

Final:  {'ang': 13, 'hap': 14, 'neu': 143, 'sad': 97}
Overall:  {'ang': 48, 'hap': 81, 'neu': 422, 'sad': 193}


In [20]:
predictions_validation_confusion = sess.run(tf.argmax(prediction, 1), feed_dict={X:validation_data, Y:validation_labels, keep_prob:1.0, normalization_switch: False})

In [21]:
predictions_validation_confusion

array([1, 1, 1, 2, 0, 2, 0, 3, 0, 2, 0, 2, 0, 3, 3, 1, 2, 2, 0, 2, 3, 2,
       2, 2, 0, 2, 2, 0, 0, 2, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 1, 2, 0, 0,
       2, 3, 2, 2, 3, 2, 3, 2, 3, 3, 0, 3, 3, 2, 3, 2, 0, 0, 0, 0, 0, 3,
       0, 0, 1, 0, 2, 3, 1, 0, 2, 2, 3, 0, 3, 1, 0, 0, 3, 0, 3, 3, 1, 3,
       1, 1, 0, 0, 2, 2, 3, 1, 2, 2, 0, 2, 2, 3, 2, 2, 1, 2, 0, 2, 3, 2,
       3, 2, 0, 3, 0, 2, 3, 3, 2, 1, 3, 0, 3, 2, 3, 1, 2, 2, 2, 1, 2, 2,
       2, 2, 3, 1, 1, 0, 1, 2, 1, 0, 1, 2, 0, 0, 1, 3, 3, 0, 2, 2, 0, 2,
       1, 2, 0, 3, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 3, 1, 3, 1, 3, 2, 0, 3,
       2, 3, 0, 3, 0, 3, 1, 2, 0, 3, 3, 0, 0, 2, 0, 2, 0, 3, 1, 0, 2, 2,
       2, 3, 3, 0, 0, 0, 0, 3, 1, 2, 3, 3, 3, 3, 1, 2, 3, 1, 2, 1, 2, 2,
       2, 3, 3, 3, 2, 2, 0, 3, 1, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 1, 2,
       1, 2, 2, 1, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3, 0, 0, 1, 3,
       2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 1, 2, 2,

In [22]:
true_labels = [onehot.index(1.0) for onehot in validation_labels]
true_labels

[2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,


In [23]:
print(confusion_matrix(predictions_validation_confusion, true_labels))

[[ 13  22 112  15]
 [  6  14  59  23]
 [ 23  31 143  58]
 [  6  14 108  97]]


In [24]:
#sess.close()